## Runnning with gammapy-dev/IRF_model
Fitting asimov datasets with nuisance parameters based on the different livetimes

### Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from gammapy.maps import Map
from astropy.coordinates import SkyCoord, Angle
from gammapy.modeling import Fit,  Parameters, Covariance , Parameter
from gammapy.datasets import MapDataset ,Datasets, FluxPointsDataset
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    SkyModel,
    PointSpatialModel,
    PowerLawNormSpectralModel,
    Models,
    SpatialModel,
    FoVBackgroundModel,
    PiecewiseNormSpectralModel,
)
from gammapy.estimators import TSMapEstimator, ExcessMapEstimator
from gammapy.estimators import FluxPoints, FluxPointsEstimator

from regions import CircleSkyRegion, RectangleSkyRegion
import yaml
import sys
sys.path.append('../')
import Dataset_load 
from matplotlib import rc
rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
rc("text", usetex=True)
from  Dataset_Setup import Setup, GaussianCovariance_matrix

## Read dataset

In [2]:
c = Dataset_load.load_config()
awo, aw, ewo, ew = c['_colors']  

In [3]:
livetimes = c['livetimes']
zero = c['zero'] 
norm = c['norm'] 
tilt = c['tilt'] 
bias = c['bias'] 
resolution = c['resolution'] 
magnitude = c['magnitude'] 
corrlength = c['corrlength']
sys = c['sys'] 
folder = c['folder']
nbidx = 0
sys
livetimes = livetimes#[7:8]
onehundred = False
if "100" in sys:
    onehundred = True

In [4]:
sys

'Combined100'

In [5]:
e_reco_n = 30

def set_up(dataset_input):
    
    setup = Setup(dataset_input=dataset_input)
    #setup.set_up_irf_sys(bias, resolution, norm, tilt)
    dataset_asimov, dataset_asimov_N = setup.run()
    # irf model
    setup.set_irf_model(dataset_asimov_N)
        
    if  "Eff_area" in sys:
        dataset_asimov_N.models.parameters['resolution'].frozen = True
        dataset_asimov_N.irf_model.parameters['tilt'].frozen = False
        dataset_asimov_N.irf_model.parameters['bias'].frozen = True
        setup.set_irf_prior(dataset_asimov_N, bias, resolution, norm, tilt)
        print(dataset_asimov_N.irf_model)
        
    if sys == "E_reco":
        dataset_asimov_N.models.parameters['resolution'].frozen = True
        dataset_asimov_N.irf_model.parameters['tilt'].frozen = True
        dataset_asimov_N.irf_model.parameters['norm'].frozen = True
        dataset_asimov_N.irf_model.parameters['bias'].frozen = False
        setup.set_irf_prior(dataset_asimov_N, bias, resolution, norm, tilt)
        e_reco_n = 3000

    if  "Combined" in sys:
        dataset_asimov_N.models.parameters['resolution'].frozen = True
        dataset_asimov_N.irf_model.parameters['tilt'].frozen = False
        dataset_asimov_N.irf_model.parameters['bias'].frozen = False
        setup.set_irf_prior(dataset_asimov_N, bias, resolution, norm, tilt)
        e_reco_n = 3000
    
    if sys == "BKG":
        
        # piece wise model
        # remove old bkg model
        setup.set_up_bkg_sys_V( breake = 10,
                            index1 = 2,
                            index2 = 1.5, 
                            magnitude = magnitude )
        
        dataset_asimov, dataset_asimov_N = setup.run()
        
        setup.unset_model(dataset_asimov_N, FoVBackgroundModel)
        setup.set_piecewise_bkg_model(dataset_asimov_N)
        # energy of the following parameters smaller than ethrshold
        dataset_asimov_N.background_model.parameters['norm0'].frozen = True
        dataset_asimov_N.background_model.parameters['norm1'].frozen = True
        dataset_asimov_N.background_model.parameters['norm2'].frozen = True
        dataset_asimov_N.background_model.parameters['norm3'].frozen = True
        setup.set_bkg_prior(dataset_asimov_N, magnitude, corrlength)
        frozen_pos = 1
        if frozen_pos:
            dataset_asimov.models.parameters['lon_0'].frozen = True
            dataset_asimov.models.parameters['lat_0'].frozen = True
            dataset_asimov_N.models.parameters['lon_0'].frozen = True
            dataset_asimov_N.models.parameters['lat_0'].frozen = True

    return dataset_asimov_N, dataset_asimov, setup

In [6]:
%%time
datasets_N = []
datasets= []
datasets_input = []
for i,l in enumerate(livetimes):
    datasets_i  = Dataset_load.create_asimov(model = c['model'], source = c['source'], 
                                                   livetime = f"{l}-hr",
                                            parameters = None)
    
    dataset_asimov_N, dataset_asimov, setup = set_up(datasets_i)
    dataset_asimov_N.e_reco_n =e_reco_n
    datasets.append( dataset_asimov)
    datasets_N.append( dataset_asimov_N)
    datasets_input.append(datasets_i)
    fitting = 0
    if fitting:
        print(i, livetimes[i])
        fit_cor = Fit(store_trace=0)
        minuit_opts = {"tol": 0.1, "strategy": 2}
        fit_cor.backend = "minuit"
        fit_cor.optimize_opts = minuit_opts
        result_cor = fit_cor.run(dataset_asimov)
        print(result_cor)
        print("saving")
        path = f'../{folder}/data/0_model_livetime_{livetimes[i]}.yml'
        dataset_asimov.models.write(path,overwrite=True)
        fig = plt.figure()
        dataset_asimov.models.covariance.plot_correlation()
        fig = plt.gcf()
        fig.savefig(f'../{folder}/plots/0_cov_livetime_{livetimes[i]}.pdf')
        plt.close()

    else:
        print(i)
        path = f'../{folder}/data/0_model_livetime_{livetimes[i]}.yml'
        dataset_asimov.models = Models.read(path)
        print(path[:-4])
        print(dataset_asimov.models)

    fitting_N = 0

    if fitting_N:
        print(i)
        fit_cor = Fit(store_trace=0)
        minuit_opts = {"tol": 0.01, "strategy": 2}
        fit_cor.backend = "minuit"
        fit_cor.optimize_opts = minuit_opts
        result_cor = fit_cor.run(dataset_asimov_N)
        print(result_cor)
        print("saving")
        path = f'../{folder}/data/0_model_nui_livetime_{livetimes[i]}_{e_reco_n}.yml'
        dataset_asimov_N.models.write(path,overwrite=True)
        dataset_asimov_N.models.covariance.plot_correlation()
        fig = plt.gcf()
        fig.savefig(f'../{folder}/plots/0_cov_nui_livetime_{livetimes[i]}.pdf')
        plt.close()

    else:
        print(livetimes[i])
        path = f'../{folder}/data/0_model_nui_livetime_{livetimes[i]}_{e_reco_n}.yml'
        print(path)
        try:
            dataset_asimov_N = Dataset_load.load_dataset_N(dataset_asimov_N, path,bkg_sys = True,
                                                   energy = dataset_asimov._geom.axes[0].center)   
        except:
            dataset_asimov_N = Dataset_load.load_dataset_N(dataset_asimov_N, path,bkg_sys = False,
                                                   energy = dataset_asimov._geom.axes[0].center)   
            dataset_asimov_N.models.parameters['tilt'].frozen = False 


    

KeyboardInterrupt: 

In [7]:
models_asimov, models_asimov_N = [], []
for i,l in enumerate(livetimes):
    models_asimov
    path = f'../{folder}/data/0_model_nui_livetime_{livetimes[i]}_{e_reco_n}.yml'
    models_asimov_N.append(Models.read(path))
    path = f'../{folder}/data/0_model_livetime_{livetimes[i]}.yml'
    models_asimov.append(Models.read(path))
    

FileNotFoundError: [Errno 2] No such file or directory: '../Combined100_PKSflare_crab_cutoff/data/0_model_nui_livetime_0.01_30.yml'

In [ ]:
def plot_residuals_spec():
    fig,axs = plt.subplots(len(datasets), 1, figsize = (6, 5 * len(datasets)))
    if len(datasets) ==1:
        axs = np.array([axs])
    axs= axs.flatten()
    
    energy_power = 2
    for i, d in enumerate(datasets):
        ax = axs[i]
        live = livetimes[i]
        dataset = datasets[i]
        dataset_N = datasets_N[i]
        dataset.plot_residuals_spectral(ax  = ax, color = awo[0], method = 'diff/sqrt(model)',
                                       capsize = 2, label = "no sys")
        dataset_N.plot_residuals_spectral(ax  = ax, color = aw[0], method = 'diff/sqrt(model)',
                                             capsize = 2, marker = 'x', label = "sys")
        ax.legend()
    return axs
        
#axs = plot_residuals_spec()
#axs[0].set_ylim(-5, 5)

In [ ]:
datasets_N[0].models[0]

In [ ]:
datasets[0].models[0]

In [ ]:
labels = [str(int(l)) for l in livetimes]
models_list_irf = [ d[-1] for d in models_asimov_N]
models_list_piecebkg = [ d[-2] for d in models_asimov_N]

pars =   [('bias', 'resolution'), ('norm', 'tilt')]

colors_default =  plt.rcParams['axes.prop_cycle'].by_key()['color'] 
colors_default = colors_default* 5

In [ ]:
i = 0
if models_list_irf[0] is not None:
    fig,axs = plt.subplots(2,1)
    for l, m in zip(livetimes, models_list_irf):
        axs[0].plot(l, m.parameters[pars[0][0]].error, 'x', label = "bias", color = colors_default[i])
        axs[0].plot(l, m.parameters[pars[0][1]].error, 'o' , label = "resolution", color = colors_default[i])
        axs[1].plot(l, m.parameters[pars[1][0]].error, 'x' , label = "norm", color = colors_default[i])
        axs[1].plot(l, m.parameters[pars[1][1]].error, 'o' , label = "tilt", color = colors_default[i])
        i +=1


    axs[0].plot([livetimes[0],livetimes[-1]], [bias, bias])
    axs[0].plot([livetimes[0],livetimes[-1]], [resolution, resolution])
    axs[1].plot([livetimes[0],livetimes[-1]], [norm, norm])
    axs[1].plot([livetimes[0],livetimes[-1]], [tilt, tilt])  

    for a in axs:
        a.set_xscale("log")
        a.legend()


        from collections import OrderedDict

        handles, labels = a.get_legend_handles_labels()
        by_label = OrderedDict(zip(labels, handles))
        a.legend(by_label.values(), by_label.keys())

    #axs[0].set_ylim(-0.01, 0.2)
    fig.savefig(f'../{folder}/plots/{nbidx}_irf_fit_results.pdf')

else:
    for i, m in enumerate(models_list_piecebkg):
        if "norm0" in m.parameters.names:
            
            plt.plot(m.spectral_model.energy.value, m.spectral_model.norms, 
                    label = livetimes[i])
            plt.xscale("log")
            plt.xlabel("Energy [TeV]")
            plt.legend()
            plt.ylabel("Fitted bkg sys")
            
    plt.plot(setup.dataset_helper.background_model.spectral_model.energy,
         setup.dataset_helper.background_model.spectral_model.norms, 
             color = 'black')
    plt.xscale("log")


In [ ]:
def plot_spectra_empty():
    fig,axs = plt.subplots(len(datasets), 1, figsize = (6, 5 * len(datasets)))
    if len(datasets) ==1:
        axs = np.array([axs])
    axs= axs.flatten()
    
    energy_power = 2
    for i, d in enumerate(datasets):
        ax = axs[i]
        live = livetimes[i]
        dataset = datasets[i]
        dataset_N = datasets_N[i]
        datasets_i.models[0].spectral_model.plot((0.4, 100) *u.TeV, energy_power = energy_power,
                                                     color = 'black', ax = ax)  
        models_asimov_N[i][0].spectral_model.plot((0.4, 100) *u.TeV, energy_power = energy_power,
                                                     color = aw[0], ax = ax)   
        
        par = models_asimov_N[i].parameters['amplitude']
        per = par.error /par.value
        models_asimov_N[i][0].spectral_model.plot_error((0.4, 100) *u.TeV,energy_power = energy_power, ax = ax, 
                                                   facecolor= aw[1], alpha = 0.8, 
                                                     label = f'With nui par.\n amplitude error =  {per*100:.3}\% ')
        models_asimov[i][0].spectral_model.plot((0.4, 100) *u.TeV,energy_power = energy_power, ax = ax, 
                                                   color= awo[0], linestyle = 'dashed')
        par = models_asimov[i].parameters['amplitude']
        per = par.error /par.value
        models_asimov[i][0].spectral_model.plot_error((0.4, 100) *u.TeV,energy_power = energy_power, ax = ax, 
                                                   facecolor= awo[1], alpha = 0.8,
                                                   label = f'W/o nui par.\n amplitude error =  {per*100:.3}\%')
        ax.set_title(f"Livetime: {float(live):.2} hr")
        ax.set_ylim(1e-14, 1e-10)

        #if i <2:
        #    ax.set_ylim(6e-13, 6e-9)
            
        ax.legend()
    plt.tight_layout()
    return fig
        
fig = plot_spectra_empty()
fig.savefig(f'../{folder}/plots/{nbidx}_spectra.pdf')


In [ ]:
np.sqrt(15.4**2 - 6.73 ** 2)

In [ ]:
irf_model = models_asimov_N[0][2].eff_area_model.spectral_model.copy()
irf_model.parameters['norm'].value  = 0.1
irf_model.parameters['tilt'].value  = 0.02
irf_model.parameters['reference'].value  = 1

ax = irf_model.plot((0.4, 100 ) *u.TeV, label = "E0 = 1 TeV")
#irf_model.plot_error((0.4, 100 ) *u.TeV)
ax.set_yscale("linear")
print( "E0 = 1 TeV")
print(f"{irf_model.evaluate(norm = 0.1, tilt = 0.02, reference = 100*u.TeV, energy = 0.4*u.TeV):.4}  sys @ {0.4*u.TeV} ")
print(f"{irf_model.evaluate(norm = 0.1, tilt = 0.02, reference = 100*u.TeV, energy = 1*u.TeV):.4}  sys @ {1*u.TeV} ")
print(f"{irf_model.evaluate(norm = 0.1, tilt = 0.02, reference = 100*u.TeV, energy = 100*u.TeV):.4}  sys @ {100*u.TeV} ")


irf_model = models_asimov_N[0][2].eff_area_model.spectral_model.copy()
irf_model.parameters['norm'].value  = 0.1
irf_model.parameters['tilt'].value  = 0.02

ax = irf_model.plot((0.4, 100 ) *u.TeV, label = "E0 = 100 TeV")
#irf_model.plot_error((0.4, 100 ) *u.TeV)
ax.set_yscale("linear")
print()
print( "E0 = 100 TeV")
print(f"{irf_model.evaluate(norm = 0.1, tilt = 0.02, reference = 100*u.TeV, energy = 0.4*u.TeV):.4}  sys @ {0.4*u.TeV} ")
print(f"{irf_model.evaluate(norm = 0.1, tilt = 0.02, reference = 100*u.TeV, energy = 1*u.TeV):.4}  sys @ {1*u.TeV} ")
print(f"{irf_model.evaluate(norm = 0.1, tilt = 0.02, reference = 100*u.TeV, energy = 100*u.TeV):.4}  sys @ {100*u.TeV} ")
ax.legend();


In [ ]:
pos = datasets_N[7].copy(name = datasets_N[7].name)
pos.models = datasets_N[7].models.copy()
pos.irf_model.parameters['norm'].value = 0.5
pos.irf_model.parameters['norm'].frozen = True
pos.irf_model.parameters['tilt'].value = 0.2
pos.irf_model.parameters['tilt'].frozen = True
plt.plot(pos.npred_signal("Crablog").get_spectrum().data.flatten(), color = 'blue')
print(pos.models)
re = Fit().run(pos)
plt.plot(pos.excess.get_spectrum().data.flatten(), marker = 'x')
plt.plot(pos.npred_signal("Crablog").get_spectrum().data.flatten(), color = 'black')
plt.yscale("log")
print(pos.models)

In [ ]:
pos

In [ ]:
ax = pos.models[0].spectral_model.plot((0.4, 100) *u.TeV, energy_power = 2, label = f"POS: {pos.models[0].parameters['index'].value:.4}, {pos.models[0].parameters['amplitude'].value:.4}")
datasets_N[7].models[0].spectral_model.plot((0.4, 100) *u.TeV, ax =ax, energy_power = 2, label = f"True: {datasets_N[7].models[0].parameters['index'].value:.4}, {datasets_N[7].models[0].parameters['amplitude'].value:.4}")
ax.legend()

In [ ]:

pos.irf_model.parameters['norm'].value = 0.
plt.plot(pos.npred_signal("Crablog").get_spectrum().data.flatten())
plt.plot(pos.excess.get_spectrum().data.flatten(), marker = 'x')
pos.irf_model.parameters['norm'].value = 0.5
plt.plot(pos.npred_signal("Crablog").get_spectrum().data.flatten(), color = 'black')
plt.yscale("log")

In [ ]:
res = Fit().run(pos)

In [ ]:
plt.plot(pos.excess.get_spectrum().data.flatten(), marker = 'x')
plt.plot(pos.npred_signal("Crablog").get_spectrum().data.flatten(), color = 'black')
plt.yscale("log")

In [ ]:
ax = pos.models[0].spectral_model.plot((0.4, 100) *u.TeV, energy_power = 2, label = f"POS: {pos.models[0].parameters['index'].value:.4}, {pos.models[0].parameters['amplitude'].value:.4}")
neg.models[0].spectral_model.plot((0.4, 100) *u.TeV, ax =ax, energy_power = 2, label = f"NEG: {neg.models[0].parameters['index'].value:.4}, {neg.models[0].parameters['amplitude'].value:.4}")
datasets_N[7].models[0].spectral_model.plot((0.4, 100) *u.TeV, ax =ax, energy_power = 2, label = f"True: {datasets_N[7].models[0].parameters['index'].value:.4}, {datasets_N[7].models[0].parameters['amplitude'].value:.4}")
ax.legend()

In [ ]:
compute = 1
if compute:
    stat_0 = []
    for d in datasets:
        s = d.stat_sum()
        with d.models.parameters.restore_status():
            d.models.parameters['amplitude'].value = 0 
            d.models[0].parameters.freeze_all()
            bkg = FoVBackgroundModel(dataset_name=d.name)
            d.models= Models([bkg])
            fit = Fit()
            result= fit.optimize(d)
            print(result)
            stat_0.append(np.sqrt( d.stat_sum() - s))
        print(stat_0)
    np.savetxt( f'../{folder}/data/0_sqrtts.txt',  np.array(stat_0))
else:
    stat_0 = np.loadtxt( f'../{folder}/data/0_sqrtts.txt')
stat_0

In [ ]:
from scipy.optimize import curve_fit
if len(stat_0)>1:
    # Given data
    x_values =livetimes
    y_values =stat_0

    # Define the linear function
    def linear_fit(x, a, b):
        return a * x**b

    # Perform the curve fit
    params, covariance = curve_fit(linear_fit, x_values, y_values)
    a_fit, b_fit = params

    # Create a log-log plot
    plt.figure(figsize=(8, 6))
    plt.loglog(x_values, y_values, 'bo', label='Data Points')
    plt.loglog(x_values, linear_fit(x_values, a_fit, b_fit), 'r--', label=f'Linear Fit: y = {a_fit:.2f} {b_fit:.2f}')
    plt.xlabel('Livetime [hr]')
    plt.ylabel('Sqrt(TS)')
    plt.legend()
    plt.show()


    def compute_x_for_given_y(y_value, a, b):
        """
        Computes the x value for a given y value using the linear fit equation.

        Args:
            y_value (float): The desired y value.
            a (float): Coefficient 'a' from the linear fit equation.
            b (float): Coefficient 'b' from the linear fit equation.

        Returns:
            float: The corresponding x value.
        """
        # Solve for x using the inverse of the linear fit equation
        x_value = (y_value / a) ** (1 / b)
        return x_value

    # Example usage
    desired_y = 5  # Replace with your desired y value
    computed_5 = compute_x_for_given_y(desired_y, a_fit, b_fit)
    print(f"For a desired sig of {desired_y:.2f}, the corresponding livetime is approximately {computed_5:.6f}.")
    desired_y = 50  # Replace with your desired y value
    computed_50 = compute_x_for_given_y(desired_y, a_fit, b_fit)
    print(f"For a desired sig of {desired_y:.2f}, the corresponding livetime is approximately {computed_50:.6f}.")


In [ ]:
fontsize = 14

inputvalues = {}
for p in datasets_input[0].models[0].parameters:
    inputvalues[p.name]  = p.value

def plotting(parname = "amplitude", delta = 1):
    fig = plt.figure(figsize  = (8,5))
    inputvalue = inputvalues[parname]
    
    errs = []
    errs_N = []
    livetimes_ext = np.array(livetimes).copy(); livetimes_ext2 = np.array(livetimes).copy(); 
    livetimes_ext[-1] *=1.1;
    livetimes_ext2[0:,] *=1.1; 
    livetimes_ext2 = np.insert(livetimes_ext2, 0, values =  livetimes_ext2[0]* 0.9)


    for i, d in enumerate(datasets_N):
        par = datasets[i].models[0].parameters[parname]
        val, err = par.value, par.error
        errs.append( err/ inputvalue )
        plt.errorbar(livetimes[i], val/ inputvalue, yerr =  err/ inputvalue,  fmt='x', 
                     color = awo[0])

        par = d.models[0].parameters[parname]
        val, err = par.value, par.error
        errs_N.append( err/ inputvalue)
        plt.errorbar( 1.1* livetimes[i] , val/ inputvalue, yerr = err/ inputvalue,  fmt='x', 
                     color = aw[0])

    

    errs_N = np.insert(errs_N, 0, values =  errs_N[0])
    plt.fill_between(  livetimes_ext2, 1- np.array(errs_N), 1+np.array(errs_N) 
                     ,color = aw[1], alpha = 0.8, label = '')
    plt.fill_between( livetimes_ext, 1- np.array(errs), 1+np.array(errs)
                     ,color = awo[1], alpha = 0.8, label = '')

    plt.errorbar( np.nan, np.nan , yerr = np.nan, fmt='x', color = aw[0],
                 label = 'fit with systematics')
    plt.errorbar( np.nan, np.nan , yerr = np.nan, fmt='x', color = awo[0], 
                 label = 'fit w/o systematics')
    ax1 = plt.gca()
    xlim = ax1.get_xlim()
    plt.hlines(1, xlim[0], xlim[1], color = 'grey')
    #plt.title("Best Fit Amplitude")
    plt.legend(fontsize = fontsize)
    plt.xscale('log')
    plt.xlabel("Livetime [hr]", fontsize = fontsize)
    plt.ylabel(f"Best-fit {parname} \n[Input]", fontsize = fontsize)

    ax2 = ax1.twiny()
    ax2.set_xscale("log")
    npred = [d.npred_signal().data.sum() for d in datasets]
    
    ax2.set_xlim(ax1.get_xlim())
    ax2.set_xticks(livetimes)
    clabels = [ f'{int(c)}' for c in npred]
    ax2.set_xticklabels(clabels)
    ax2.set_xlabel(r"Signal Counts", fontsize = fontsize)
    plt.tight_layout()
    ax = plt.gca()
    ax.set_ylim(1- delta, 1. + delta)
    sigma_color = 'black'
    ax.vlines(computed_50, ax.get_ylim()[0], ax.get_ylim()[1], color = sigma_color,linestyle =  'dashed')
    ax.vlines(computed_5, ax.get_ylim()[0], ax.get_ylim()[1], color = sigma_color,linestyle =  'dotted')
    ax.text(computed_50*1.05, ax.get_ylim()[1]*0.9 ,"50 $\sigma$", color = sigma_color, fontsize = fontsize)
    ax.text(computed_5*1.05, ax.get_ylim()[1]*0.9 ,"5 $\sigma$", color = sigma_color, fontsize = fontsize)
    return fig

In [ ]:
fig = plotting(parname = "amplitude", delta = 0.9)
fontsize = 14
fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_amplitude.svg')
fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_amplitude.pdf')
fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_amplitude.png')

In [ ]:
parname = "index"
fig = plotting(parname = parname, delta = 0.2)

fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.svg')
fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.pdf')
fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.png')

In [ ]:
try:
    parname = "lambda_"
    fig = plotting(parname = parname, delta = 3)


    fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.svg')
    fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.pdf')
    fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.png')
except:
    print("no cutoff")

In [ ]:
try:
    parname = "sigma"
    fig = plotting(parname = parname, delta = 3)


    fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.svg')
    fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.pdf')
    fig.savefig(f'../{folder}/plots/{nbidx}_best_fit_vs_livetime_{parname}.png')
except:
    print("no cutoff")

## BKG

In [ ]:
sys

In [ ]:
if sys == "BKG":
    i = 7
    fig, ax = plt.subplots(1,1)
    method = 'diff'
    datasets[i].plot_residuals_spectral(ax  = ax, color = awo[0], method = method,
                                       capsize = 2, label = "no sys")
    datasets_N[i].plot_residuals_spectral(ax  = ax, color = aw[0], method = method,
                                             capsize = 2, marker = 'x', label = "sys")

## RND Fluxpoints

In [ ]:
for i, d in enumerate(datasets):
    l = livetimes[i]
    print()
    print(l)
    dataset = datasets[i]
    dataset_N = datasets_N[i]
    parameters = dataset.models[0].parameters.names
    for p in parameters:
        
        par = dataset.models[0].parameters[p]
        per = par.error /par.value * 100 
        print(f"{p}:")
        print(f"without {per:.4} %")
        par = dataset_N.models[0].parameters[p]
        per = par.error /par.value * 100 
        
        print(f"with    {per:.4} %")        

In [ ]:
datasets_N[7].models.covariance.plot_correlation()

In [ ]:
par_names = ['index', 'amplitude', 'lambda_', 'bias', 'resolution', 'norm' , 
            'tilt']
names =  ['index', 'amplitude',  
        'bkg norm', 'bkg tilt',
          'bias', 'resolution',
          'eff. area norm' , 
            'eff. area tilt']
if fitting_N:
    for i in range(len(livetimes)):

        #datasets_N[i].models.read_covariance("data", f"8_model_nui_livetime_{livetimes[i]}_covariance.dat", format="ascii.fixed_width")
        parameters = Parameters([_ for _ in datasets_N[i].models.parameters if _.name in par_names])
        cov_par = datasets_N[i].models.covariance.get_subcovariance(parameters)
        ax = cov_par.plot_correlation()
        ax.set_title(str(livetimes[i]) + " hr")
        fig = plt.gcf()
        fig.savefig(f'../{folder}/plots/{nbidx}_covmatrix_livetime_{livetimes[i]}.svg')
        fig.savefig(f'../{folder}/plots/{nbidx}_covmatrix_livetime_{livetimes[i]}.pdf')



In [ ]:
for d in datasets:
    d.models.covariance.plot_correlation()

In [ ]:
for d in datasets_N:
    d.models.covariance.plot_correlation()